# 95/20 Ranking

Estimate top-coded incomes from PUMs data. It sounds like we should pull with a PUMA instead of MET2013, and translate the geography in the analysis.

In [ ]:
import pandas

from_census = pandas.read_csv("too_big/csv_hct/ss14hct.csv") ## Grab for every topcoded state

pums = pandas.read_csv("too_big/usa_00001.csv")
acs_14 = pandas.read_csv("hh_14.csv",encoding = "ISO-8859-1")

In [ ]:
census_vars = ["PUMA","ADJINC","SERIALNO","HINCP","WGTP"]

#bridgeport_pumas = ["00100", "00101", "00102", "00103", "00104", "00105"]

from_census["bridgeport"] = from_census["PUMA"].apply(lambda x: ((x>=100) & (x<=105)))

bridgeport_2014 = from_census[from_census["bridgeport"]==True][census_vars]
bridgeport_2014["HHINC"] = pandas.to_numeric(bridgeport_2014["HINCP"],errors="coerce")
bridgeport_2014["ADJHH"] = bridgeport_2014.apply(lambda x: (x["ADJINC"]/1000000)*x["HHINC"], axis=1)

In [ ]:
def multiply(income,weight,tar):
    i = 0
    while i < weight:
        tar.append(income)
        i+=1

def explode_weights(df,income_col, weight_col):
    ## Make a list of HHINCOME for calculating
    ## a quantile where each household's HHINCOME
    ## is repeated in the series n times, where
    ## n == weight_col
    tar = []
    df.apply(lambda x: multiply(x[income_col],x[weight_col],tar),axis=1)
    return tar

exploded_bridgeport = explode_weights(bridgeport_2014,"ADJHH","WGTP")

In [ ]:
pandas.Series(exploded_bridgeport).quantile(.95, interpolation="lower")

# Hallelujah!

This is the number Brookings had! But from talking to the Census it sounds like they were wrong to adjust income to constant dollars, so we won't do that for our analysis:

In [ ]:
exploded_bridgeport = explode_weights(bridgeport_2014,"HHINC","WGTP")
pandas.Series(exploded_bridgeport).quantile(.95)

# What's left:

#1 Write the functions to streamline it
#2 Gather the necessary 2016 data: ACS & PUMS
#3 Put them where they need to be & fill out the 2016 data set
#4 Do the analysis for 2007, too, and get the over-time chart (or maybe figure out when EPI measured it's recession numbers from?)

* identify all the topcoded entries -- get the right state PUMs files ugh such a pain

* find the corresponding PUMAs in MABLE/GEOCORR, and figure out what to do if any of them cross a CBSA border

* extract the correct data from the PUMs files... a lot of this might be manual

In [ ]:
def get_quantile(df,column,weight=None,quant):
    if weight is not None:
        df = explode_weights(df,column,weight)
    return df.quantile(quant)

In [ ]:
acs_16 = pandas.read_csv("hh_16.csv", encoding = "ISO-8859-1")
mable = pandas.read_csv("geocorr15.csv", encoding = "ISO-8859-1")[1:]

top_coded = acs_16[acs_16["HD01_VD06"].isnull()]
bottom_coded = acs_16[acs_16["HD01_VD02"].isnull()]

top_coded["States"] = top_coded["GEO.display-label"].apply(lambda x: x.split(',')[1].split()[0].split('-'))
#top_coded["States"] = top_coded["States"].drop_duplicates()

pandas.Series(top_coded["States"].sum()).drop_duplicates().sort_values()

In [ ]:
mable.head()

In [ ]:
mable["cbsa"] = mable["cbsa"].apply(lambda x : pandas.to_numeric(x,errors="coerce"))

def get_pumas(geo_id,geo_corr):
    cbsa = geo_corr[geo_corr["cbsa"] == geo_id] 
    if len(cbsa) == 0:
        print(geo_id)
    target = {}
    for index,row in cbsa.iterrows(): # do i need "index"?
        target[row["puma12"]] = row["afact"]
    return target

top_coded["PUMAS"] = top_coded["GEO.id2"].apply(lambda x: get_pumas(x,mable))

In [ ]:
mable["cbsa10"].unique()

In [96]:
top_coded

,GEO.id,GEO.id2,GEO.display-label,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,HD01_VD06,HD02_VD06,States,PUMAS
0,1600000US0640354,640354,"La Quinta city, California",27794.0,2167.0,55920,4923,90161,6006,146206,6942,NaN,NaN,[California],{}
109,310M300US14460,14460,"Boston-Cambridge-Newton, MA-NH Metro Area",29225.0,345.0,60545,343,98251,581,154820,789,NaN,NaN,"[MA, NH]","{'00400': '0.346 ', '00501': '1 ', '00502': '1..."
110,310M300US14500,14500,"Boulder, CO Metro Area",28368.0,911.0,55378,1196,91056,1521,148663,2650,NaN,NaN,[CO],"{'00801': '0.211 ', '00802': '0.704 ', '00803'..."
120,310M300US14860,14860,"Bridgeport-Stamford-Norwalk, CT Metro Area",32385.0,674.0,66551,945,110368,1393,191602,2122,NaN,NaN,[CT],"{'00100': '1 ', '00101': '1 ', '00102': '1 ', ..."
259,310M300US20780,20780,"Edwards, CO Micro Area",40478.0,2582.0,66183,4343,94777,5378,139984,3936,NaN,NaN,[CO],{'00400': '0.46 '}
417,310M300US27220,27220,"Jackson, WY-ID Micro Area",32402.0,5774.0,59526,4904,85468,5229,129621,9133,NaN,NaN,"[WY, ID]","{'01100': '0.078 ', '00100': '0.196 '}"
506,310M300US31060,31060,"Los Alamos, NM Micro Area",47250.0,8691.0,88420,3310,123242,8281,177485,9948,NaN,NaN,[NM],{'00300': '0.143 '}
507,310M300US31080,31080,"Los Angeles-Long Beach-Anaheim, CA Metro Area",24426.0,149.0,48258,212,79542,294,130673,456,NaN,NaN,[CA],"{'03701': '1 ', '03702': '1 ', '03703': '1 ', ..."
563,310M300US33260,33260,"Midland, TX Metro Area",31622.0,922.0,57037,2005,89892,3126,140136,4595,NaN,NaN,[TX],"{'02800': '0.047 ', '03000': '1 '}"
601,310M300US34900,34900,"Napa, CA Metro Area",32283.0,1433.0,59536,1913,93005,2113,146169,3402,NaN,NaN,[CA],{'05500': '1 '}


# Qunitile Wages 2007-2016

Track either mean wages per quintile or quintile cut-offs over time, to show the stratification of the recession and the recovery. Probably want to do this for something like:

* CT (state)
* CT (urban areas)
* Bridgeport Metro Area
* New Haven Metro Area
* Bridgeport as city

OR do this or mean income urban vs non-urban w/in Connecticut?

So you need a call for each year -- and the url construction might change year to year, though I think it's mostly just 2016 that's special for some reason.

A bunch of maybe-useful api queries:

    * https://api.census.gov/data/2016/acs/acs1/variables.html
    * https://api.census.gov/data/2016/acs/acs1/examples.html
    * https://api.census.gov/data/2016/acs/acs1?get=B01001_001E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*
    * https://api.census.gov/data/2015/acs1/examples.html
    * https://api.census.gov/data/2016/
            

In [ ]:
import json,urllib,re,numpy,time,config

base = "https://api.census.gov/data/"

def construct_url(year,geo_type,geo_code):
    url = base+str(year)+"/acs/acs1?get="+config.fec_key+"&is_amended=False&per_page=100"
    return(url)

def lookup(cid, t=5):
    url = construct_url(cid)
    data = url_to_data(url)
    if t>0:
        time.sleep(t)
    if data is None:
        return None
    return data["results"]

## If the fn can't find a response at the url, it returns None
## That gets converted into an empty dataframe (not ideal), and
## and flagged for the second pass
def url_to_data(url):
    try:
        req = urllib.request.Request(url)
        req.add_header('User-agent', 'Mozilla 5.10')
        res = urllib.request.urlopen(req)
        data = json.load(res)
        return data
    except urllib.error.HTTPError as err:
        return None
    
lookup("C00406389")